In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

In [2]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_ML\FeaturesLocal\FeaturesSC"

## <span style="color:yellow"> **Preparamos el set de entramiento para sacar features** </span> 

In [3]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_ML\Training Sets"

In [4]:
ins_trn = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); ins_trn["_sc"] = pd.to_datetime(ins_trn["date"]) - dt.datetime( year = 2019, month = 4, day = 18 ); ins_trn["_sc"] = ins_trn["_sc"].dt.total_seconds();
ins_tst = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); ins_tst["_sc"] = pd.to_datetime(ins_trn["date"]) - dt.datetime( year = 2019, month = 4, day = 21 ); ins_tst["_sc"] = ins_tst["_sc"].dt.total_seconds(); 

In [5]:
ins_trn_frt = ins_trn.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_tst_frt = ins_tst.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

## <span style="color:yellow"> **Preparamos el ref_hash de cada ventana** </span> 

In [6]:
loc_lb = r"D:\FacundoTorraca\Documents\TP2_ML\Labels"

In [7]:
lb_ins_trn = pd.read_csv( loc_lb + "\\label_ins_21_23.csv" );
lb_ins_tst = pd.read_csv( loc_lb + "\\label_ins_24_26.csv" ); 

In [8]:
rh_trn = lb_ins_trn[ ["ref_hash"] ]
rh_tst = lb_ins_tst[ ["ref_hash"] ]

tg_trn = lb_ins_trn[ ["21_23_sc"] ]
tg_tst = lb_ins_tst[ ["24_26_sc"] ]

## <span style="color:yellow"> **=======================================================================================================** </span> 

## <span style="color:green"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [9]:
cant_ins_trn = rh_trn.copy()
cant_ins_tst = rh_tst.copy()

cant_ins_trn = cant_ins_trn.merge( ins_trn["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" ).fillna( 0 )
cant_ins_tst = cant_ins_tst.merge( ins_tst["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" ).fillna( 0 )

cant_ins_trn.to_csv( loc_ftr + "\\cant_ins_trn.csv", index = False )
cant_ins_tst.to_csv( loc_ftr + "\\cant_ins_tst.csv", index = False )

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [10]:
main_app_trn = rh_trn.copy()
main_app_tst = rh_tst.copy()

app_mas_ins_trn = ins_trn.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_tst = ins_tst.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_trn = app_mas_ins_trn.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_trn["cant_ins"] )
app_mas_ins_tst = app_mas_ins_tst.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_tst["cant_ins"] ) 

main_app_trn = main_app_trn.merge( app_mas_ins_trn, how = "left", on = "ref_hash" )
main_app_tst = main_app_tst.merge( app_mas_ins_tst, how = "left", on = "ref_hash" )

main_app_trn["to_count"] = 1; main_app_trn["application_id"] = main_app_trn[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_trn); del(main_app_trn["to_count"])
main_app_tst["to_count"] = 1; main_app_tst["application_id"] = main_app_tst[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_tst); del(main_app_tst["to_count"])

main_app_trn.fillna( main_app_trn.isnull().sum()/len(main_app_trn) )
main_app_tst.fillna( main_app_tst.isnull().sum()/len(main_app_tst) )

main_app_trn.to_csv( loc_ftr + "\\main_app_trn.csv", index = False )
main_app_tst.to_csv( loc_ftr + "\\main_app_tst.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [11]:
cins_imp_trn = rh_trn.copy()
cins_imp_tst = rh_tst.copy()

cins_imp_trn["cant_imp"] = ins_trn["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_trn = cins_imp_trn.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()
cins_imp_tst["cant_imp"] = ins_tst["implicit"].apply( lambda x: 1 if x else 0 ); cins_imp_tst = cins_imp_tst.groupby( "ref_hash" ).agg( {"cant_imp":"sum"} ).reset_index()

cins_imp_trn.to_csv( loc_ftr + "\\cins_imp_trn.csv", index = False )
cins_imp_tst.to_csv( loc_ftr + "\\cins_imp_tst.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [12]:
cins_atr_trn = rh_trn.copy()
cins_atr_tst = rh_tst.copy()

cins_atr_trn["cant_atr"] = ins_trn["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_trn = cins_atr_trn.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()
cins_atr_tst["cant_atr"] = ins_tst["attributed"].apply( lambda x: 1 if x else 0 ); cins_atr_tst = cins_atr_tst.groupby( "ref_hash" ).agg( {"cant_atr":"sum"} ).reset_index()

cins_atr_trn.to_csv( loc_ftr + "\\cins_atr_trn.csv", index = False )
cins_atr_tst.to_csv( loc_ftr + "\\cins_atr_tst.csv", index = False )

## <span style="color:green"> **Tiempo hasta la primera instalacion en la ventana anterior** </span>
Se define con el tiempo maximo si la persona no realizo una instalacion en la ventana anterior

In [16]:
frst_ins_trn = rh_trn.copy()
frst_ins_tst = rh_tst.copy()

time_to_first_ind_trn = ins_trn_frt[ ["ref_hash","_sc"] ]
time_to_first_ind_tst = ins_tst_frt[ ["ref_hash","_sc"] ]

frst_ins_trn = frst_ins_trn.merge( time_to_first_ind_trn, how = "left", on = "ref_hash" ).fillna( 3*3600*24 )
frst_ins_tst = frst_ins_tst.merge( time_to_first_ind_tst, how = "left", on = "ref_hash" ).fillna( 3*3600*24 )

frst_ins_trn.to_csv( loc_ftr + "\\frst_ins_trn.csv", index = False )
frst_ins_tst.to_csv( loc_ftr + "\\frst_ins_tst.csv", index = False )

## <span style="color:green"> **Instalo entre 4 hs y 10 hs (Mañana)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [17]:
ins_trn['ins_4_10'] = ( pd.to_datetime( ins_trn['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_trn['date'] ).dt.hour < 11) 
ins_tst['ins_4_10'] = ( pd.to_datetime( ins_tst['date'] ).dt.hour > 3) & ( pd.to_datetime( ins_tst['date'] ).dt.hour < 11)

ins_morn_trn = rh_trn.copy()
ins_morn_tst = rh_tst.copy()

hour_mode_trn = ins_trn.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_tst = ins_tst.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_trn = ins_morn_trn.merge( hour_mode_tst, how = "left", on = "ref_hash" )
ins_morn_tst = ins_morn_tst.merge( hour_mode_trn, how = "left", on = "ref_hash" )

ins_morn_trn["ins_4_10"] =  (ins_morn_trn["ins_4_10"] > 1).astype('int8')
ins_morn_tst["ins_4_10"] =  (ins_morn_tst["ins_4_10"] > 1).astype('int8')

ins_morn_trn.to_csv( loc_ftr + "\\ins_morn_trn.csv", index = False )
ins_morn_tst.to_csv( loc_ftr + "\\ins_morn_tst.csv", index = False )

del ins_trn['ins_4_10']
del ins_tst['ins_4_10']

## <span style="color:green"> **Instalo entre 11 hs y 15 hs (Medio dia)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [18]:
ins_trn['ins_11_15'] = ( pd.to_datetime( ins_trn['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_trn['date'] ).dt.hour < 16) 
ins_tst['ins_11_15'] = ( pd.to_datetime( ins_tst['date'] ).dt.hour > 10) & ( pd.to_datetime( ins_tst['date'] ).dt.hour < 16)

ins_midday_trn = rh_trn.copy()
ins_midday_tst = rh_tst.copy()

hour_mode_trn = ins_trn.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_tst = ins_tst.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_trn = ins_midday_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
ins_midday_tst = ins_midday_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

ins_midday_trn["ins_11_15"] =  (ins_midday_trn["ins_11_15"] > 1).astype('int8')
ins_midday_tst["ins_11_15"] =  (ins_midday_tst["ins_11_15"] > 1).astype('int8')

ins_midday_trn.to_csv( loc_ftr + "\\ins_mday_trn.csv", index = False )
ins_midday_tst.to_csv( loc_ftr + "\\ins_mday_tst.csv", index = False )

del ins_trn['ins_11_15']
del ins_tst['ins_11_15']

## <span style="color:green"> **Instalo entre 16 hs y 20 hs (Tarde)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [19]:
ins_trn['ins_16_20'] = ( pd.to_datetime( ins_trn['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_trn['date'] ).dt.hour < 21) 
ins_tst['ins_16_20'] = ( pd.to_datetime( ins_tst['date'] ).dt.hour > 15) & ( pd.to_datetime( ins_tst['date'] ).dt.hour < 21)

ins_after_trn = rh_trn.copy()
ins_after_tst = rh_tst.copy()

hour_mode_trn = ins_trn.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_tst = ins_tst.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_trn = ins_after_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
ins_after_tst = ins_after_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

ins_after_trn["ins_16_20"] =  (ins_after_trn["ins_16_20"] > 1).astype('int8')
ins_after_tst["ins_16_20"] =  (ins_after_tst["ins_16_20"] > 1).astype('int8')

ins_after_trn.to_csv( loc_ftr + "\\ins_aftr_trn.csv", index = False )
ins_after_tst.to_csv( loc_ftr + "\\ins_aftr_tst.csv", index = False )

del ins_trn['ins_16_20']
del ins_tst['ins_16_20']

## <span style="color:green"> **Instalo entre 21 hs y 3 hs (Noche)** </span> 

#### <span style="color:Orange"> **One-hot Encoding** </span> 

In [20]:
ins_trn['ins_21_3'] = ( pd.to_datetime( ins_trn['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_trn['date'] ).dt.hour < 4) 
ins_tst['ins_21_3'] = ( pd.to_datetime( ins_tst['date'] ).dt.hour > 20) | ( pd.to_datetime( ins_tst['date'] ).dt.hour < 4)

ins_night_trn = rh_trn.copy()
ins_night_tst = rh_tst.copy()

hour_mode_trn = ins_trn.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_tst = ins_tst.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_trn = ins_night_trn.merge( hour_mode_trn, how = "left", on = "ref_hash" )
ins_night_tst = ins_night_tst.merge( hour_mode_tst, how = "left", on = "ref_hash" )

ins_night_trn["ins_21_3"] =  (ins_night_trn["ins_21_3"] > 1).astype('int8')
ins_night_tst["ins_21_3"] =  (ins_night_tst["ins_21_3"] > 1).astype('int8')

ins_night_trn.to_csv( loc_ftr + "\\ins_nght_trn.csv", index = False )
ins_night_tst.to_csv( loc_ftr + "\\ins_nght_tst.csv", index = False )

del ins_trn['ins_21_3']
del ins_tst['ins_21_3']